# Errors in Anisotropy Estimation

As in every image analysis pipeline, there are several possible errors that contribute to its wrong estimation. Understanding the effects of each kind of error will help us find the adequate corrections for them. We will analyze here the following sources:

- Errors in Intensity Estimation
- Errors in shift registration

## Errors in Intensity Estimation

Adequate intensity estimation requires that the images are corrected for inhomogeneities in illumination and background. What are the effects of imperfect corrections of these sources? How are these appreciated in experiments?

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

import anisotropy_functions as af
from cell_sim import Cell, Biosensor, Microscope, Corrector

In [ ]:
@interact(I_factor=(0, 2000, 100), 
          a_dimer=(0, 0.4, 0.02), a_monomer=(0, 0.4, 0.02), 
          e_par=(-1000, 1000, 50), e_per=(-1000, 1000, 50))
def plot(I_factor=1000, a_dimer=0.22, a_monomer=0.3, e_par=-400, e_per=-400):
    monomer_fraction = 1 / (1 + np.exp(-np.arange(-50, 50)/6))
    anisotropy = af.anisotropy_from_monomer(monomer_fraction, a_monomer, a_dimer, 1)
    total_fluo = af.total_fluorescence_from_monomer(monomer_fraction, 1, I_factor)
    I_parallel = af.intensity_parallel_from_anisotropy(anisotropy, total_fluo) + e_par
    I_perpendicular = af.intensity_perpendicular_from_anisotropy(anisotropy, total_fluo) + e_per
    anisotropy_from_int = af.anisotropy_from_intensity(I_parallel, 
                                                       I_perpendicular)
    
    fig, axs = plt.subplots(2, 1, sharex=True, figsize=(5, 7))
    
    axs[0].plot(I_parallel, c='g', label='Parallel')
    axs[0].plot(I_perpendicular, c='b', label='Perpendicular')
    axs[0].legend()
    axs[0].set_ylabel('Intensity')
    
    axs[1].axhline(y=a_dimer, color='k', ls='--')
    axs[1].axhline(y=a_monomer, color='k', ls='--')
    axs[1].plot(anisotropy_from_int, color='r', label='Estimated')
    axs[1].plot(anisotropy, color='r', alpha=0.5, label='Objective')
    axs[1].set_ylabel('Anisotropy')
    axs[1].legend()

    plt.subplots_adjust(hspace=0)
    plt.show()

In [ ]:
def plot(I_factor=0.3, a_dimer=0.22, a_monomer=0.3, e_par=-400, e_per=-400):
    monomer_fraction = 1 / (1 + np.exp(-np.arange(-50, 50)/6))
    anisotropy = af.anisotropy_from_monomer(monomer_fraction, a_monomer, a_dimer, 1)
    total_fluo = af.total_fluorescence_from_monomer(monomer_fraction, 1, I_factor)
    I_parallel = af.intensity_parallel_from_anisotropy(anisotropy, total_fluo) + e_par
    I_perpendicular = af.intensity_perpendicular_from_anisotropy(anisotropy, total_fluo) + e_per
    anisotropy_from_int = af.anisotropy_from_intensity(I_parallel, 
                                                       I_perpendicular)
    
    fig, axs = plt.subplots(2, 1, sharex=True, figsize=(5, 7))
    
    axs[0].plot(I_parallel, c='g', label='Paralelo')
    axs[0].plot(I_perpendicular, c='b', label='Perpendicular')
    axs[0].plot(I_parallel - e_par, c='g', alpha=0.5, label='Real')
    axs[0].plot(I_perpendicular - e_per, c='b', alpha=0.5, label='Real')
    axs[0].legend()
    axs[0].set_ylabel('Intensidad (u.a.)')
    
    axs[1].axhline(y=a_dimer, color='k', ls='--')
    axs[1].axhline(y=a_monomer, color='k', ls='--')
    axs[1].plot(anisotropy_from_int, color='r', label='Estimado')
    axs[1].plot(anisotropy, color='r', alpha=0.5, label='Real')
    axs[1].set_ylabel('Anisotropía')
    axs[1].set_xlabel('Tiempo (min.)')
    axs[1].legend()
    plt.subplots_adjust(hspace=0)
    
    plt.savefig('error_bkg.svg', format='svg')

In [ ]:
plot(I_factor=1000, a_dimer=0.22, a_monomer=0.3, e_par=0, e_per=-10)


It is important to notice that there different monomer and dimer anisotropies are possible depending on the errors. We must highlight that in some cases monomer anisotropy might be higher than dimer anisotropy, and this looks like a reversed anisotropy curve. Aditionally, theoritacally impossible values are also possible depending on the magnitude of the errors. Furthermore, if the error in intensity has a time dependance this will affect the shape of the curve. 

## Errors in shift registration

Due to the thickness of high quality polarizers, it is likely that parallel and perpendicular images are shifted between each other. If we were to generate an anisotropy image, we need to be able to correct this shift. What happens if this is not adequately corrected? How can we bypass these problems? How can we estimate the best correction?

Let's begin by generating a simulated squared cell with a gradient of concentration of biosensors. We can choose the maximum number of biosensors expected and if we are to add poisson noise to this number.

In [ ]:
proteins = 800
poisson = True

cell = Cell(proteins, poisson)

In [ ]:
plt.imshow(cell.cell_image, interpolation='none')
plt.colorbar()
plt.axis('off')
plt.show()

Let's define an anisotropy state for the cell.

In [ ]:
anisotropy = 0.26
cell.add_biosensor({'anisotropy_monomer':0.3, 'anisotropy_dimer': 0.22, 'delta_b': 0.15})

cell.generate_intensity_images(anisotropy)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flatten()

this_im = axs[0].imshow(cell.parallel_image, interpolation='none')
fig.colorbar(this_im, ax=axs[0])

axs[0].axis('off')
axs[0].set_title('Parallel Image')

this_im = axs[1].imshow(cell.perpendicular_image, interpolation='none')
fig.colorbar(this_im, ax=axs[1])

axs[1].axis('off')
axs[1].set_title('Perpendicular Image')

axs[2].hist(cell.parallel_image[200:800, 200:800].flatten(), bins=100, log=True)
axs[2].set_title('Parallel Image Histogram')

axs[3].hist(cell.perpendicular_image[200:800, 200:800].flatten(), bins=100, log=True)
axs[3].set_title('Perpendicular Image Histogram')

plt.show()

We should estimate now the anisotropy image before adding acquisition noise.

In [ ]:
non_acquired_anisotropy_image = np.zeros_like(cell.parallel_image)
nonzeros = cell.mask
non_acquired_anisotropy_image[nonzeros] = af.anisotropy_from_intensity(cell.parallel_image[nonzeros], cell.perpendicular_image[nonzeros])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

this_im = axs[0].imshow(non_acquired_anisotropy_image, vmin=0.19, vmax=0.4, interpolation='none')
axs[0].axis('off')
fig.colorbar(this_im, ax=axs[0])

axs[1].hist(non_acquired_anisotropy_image[cell.mask].flatten(), bins=np.arange(0.2, 0.3, 0.001))
axs[1].axvline(x=anisotropy, color='k', ls='--')

plt.show()

After obtaining both intensity images, we could add some aquisition noise to the images.

In [ ]:
microscope = Microscope()

parallel_image, perpendicular_image = microscope.acquire_cell(cell)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flatten()

this_im = axs[0].imshow(parallel_image, interpolation='none')
fig.colorbar(this_im, ax=axs[0])

axs[0].axis('off')
axs[0].set_title('Parallel Image')

this_im = axs[1].imshow(perpendicular_image, interpolation='none')
fig.colorbar(this_im, ax=axs[1])

axs[1].axis('off')
axs[1].set_title('Perpendicular Image')

axs[2].hist(parallel_image.flatten(), bins=100, log=True)
axs[2].set_title('Parallel Image Histogram')

axs[3].hist(perpendicular_image.flatten(), bins=100, log=True)
axs[3].set_title('Perpendicular Image Histogram')

plt.show()

In [ ]:
anisotropy_image = np.zeros_like(parallel_image)
nonzeros = cell.mask
anisotropy_image[nonzeros] = af.anisotropy_from_intensity(parallel_image[nonzeros], perpendicular_image[nonzeros])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

this_im = axs[0].imshow(anisotropy_image, interpolation='none')
axs[0].axis('off')
fig.colorbar(this_im, ax=axs[0])

axs[1].hist(anisotropy_image[cell.mask].flatten(), bins=100)
axs[1].axvline(x=anisotropy, color='k', ls='--')

plt.show()

Now we need to add the image analysis steps and corrections we would normally implement to test them and choose the best option.

In [ ]:
corrector = Corrector()
corrected_parallel, corrected_perpendicular = corrector.correct(parallel_image, perpendicular_image)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flatten()

this_im = axs[0].imshow(corrected_parallel, interpolation='none')
fig.colorbar(this_im, ax=axs[0])

axs[0].axis('off')
axs[0].set_title('Parallel Image')

this_im = axs[1].imshow(corrected_perpendicular, interpolation='none')
fig.colorbar(this_im, ax=axs[1])

axs[1].axis('off')
axs[1].set_title('Perpendicular Image')

axs[2].hist(corrected_parallel.flatten(), bins=100, log=True)
axs[2].set_title('Parallel Image Histogram')

axs[3].hist(corrected_perpendicular.flatten(), bins=100, log=True)
axs[3].set_title('Perpendicular Image Histogram')

plt.show()

In [ ]:
corrected_anisotropy_image = np.zeros_like(corrected_parallel)
nonzeros = np.nonzero(corrected_parallel)
corrected_anisotropy_image[nonzeros] = af.anisotropy_from_intensity(corrected_parallel[nonzeros], corrected_perpendicular[nonzeros])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

this_im = axs[0].imshow(corrected_anisotropy_image, vmin=0.15, vmax=0.3, interpolation='none')
axs[0].axis('off')
fig.colorbar(this_im, ax=axs[0])

axs[1].hist(corrected_anisotropy_image[cell.mask].flatten(), bins=100, log=True)
axs[1].axvline(x=anisotropy, color='k', ls='--')

plt.show()

Up to here looks like the correction worked and generated a good anisotropy image.

What would happen if our shift correction was not perfect? What happens if our background correction is not perfect?

In [ ]:
(x_shift, y_shift) = (-3, -6)

imperfect_corrector = Corrector()
# imperfect_corrector.bkg_params ={'bkg_value': bkg_value}
imperfect_corrector.shift = (x_shift, y_shift)
corrected_parallel, corrected_perpendicular = imperfect_corrector.correct(parallel_image, perpendicular_image)

In [ ]:
corrected_anisotropy_image = np.zeros_like(corrected_parallel)
nonzeros = np.nonzero(corrected_parallel)
corrected_anisotropy_image[nonzeros] = af.anisotropy_from_intensity(corrected_parallel[nonzeros], corrected_perpendicular[nonzeros])

In [ ]:
mask = np.zeros_like(corrected_anisotropy_image).astype(bool)
mask[60:140, 60:140] = True

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

this_im = axs[0].imshow(corrected_anisotropy_image, vmin=0.15, vmax=0.3, cmap='plasma', interpolation='none')
axs[0].axis('off')
fig.colorbar(this_im, ax=axs[0])

axs[1].hist(corrected_anisotropy_image[mask].flatten(), log=True, bins=np.arange(0.1, 0.45, 0.002))
axs[1].axvline(x=anisotropy, color='k', ls='--')
axs[1].set_xlabel('Anisotropía')
axs[1].set_ylabel('Cuentas')

plt.savefig('anisotropy_variance_shifted.svg', format='svg')
plt.show()

In [ ]:
np.mean(corrected_anisotropy_image[mask].flatten())

In [ ]:
from matplotlib import colors
import matplotlib as mpl

In [ ]:
def plot(I_factor=0.3, a_dimer=0.22, a_monomer=0.3):
    monomer_fraction = 1 / (1 + np.exp(-np.arange(-50, 50)/8))
    bs = np.arange(0.7, 1.3, 0.1)
    
    fig, axs = plt.subplots(3, 1, sharex=True, figsize=(5, 8))
    cmaps = {b: plt.cm.coolwarm(colors.Normalize(vmin=bs[0], vmax=bs[-1])(b)) for b in bs}
    
    for b in bs:
        anisotropy = af.anisotropy_from_monomer(monomer_fraction, a_monomer, a_dimer, b)
        der_ani = np.diff(anisotropy)

        axs[0].plot(anisotropy, color=cmaps[b], alpha=0.7)
        
        anisotropy_normalized = anisotropy[:-1] - np.min(anisotropy)
        anisotropy_normalized = anisotropy_normalized / np.max(anisotropy_normalized)
        activity = der_ani / ((1 + (b-1) * anisotropy_normalized) ** 2)
        
        axs[1].plot(der_ani / np.max(der_ani), color=cmaps[b], alpha=0.7)
        
        axs[2].plot(activity / np.max(activity), color=cmaps[b], alpha=0.7)
    
    axs[0].axhline(y=a_dimer, color='k', ls='--')
    axs[0].axhline(y=a_monomer, color='k', ls='--')
    axs[1].axhline(y=0, color='k')
    axs[2].axhline(y=0, color='k')
        
    axs[0].set_ylabel('Anisotropía')
    axs[1].set_ylabel('Derivada de Anisotropía')
    axs[2].set_ylabel('Actividad')
    axs[2].set_xlabel('Tiempo (min.)')
    plt.subplots_adjust(hspace=0)
    
#     col_map = plt.get_cmap('coolwarm')
#     mpl.colorbar.ColorbarBase(axs[0], cmap=col_map, orientation = 'vertical')

    # As for a more fancy example, you can also give an axes by hand:
#     c_map_ax = fig.add_axes([0.2, 0.8, 0.6, 0.02])
#     c_map_ax.axes.get_xaxis().set_visible(False)
#     c_map_ax.axes.get_yaxis().set_visible(False)

    # and create another colorbar with:
#     mpl.colorbar.ColorbarBase(c_map_ax, cmap=col_map, orientation = 'horizontal')
    
    plt.savefig('sweep_b.svg', format='svg')
    plt.show()

In [ ]:
plot(1000, 0.22, 0.3)

In [ ]:
bs = np.arange(0.7, 1.3, 0.1)
plt.imshow(np.stack(np.array([np.array(bs)]*7)), cmap='coolwarm', alpha=0.7)

plt.colorbar()

plt.savefig('just_colorbar.svg', bbox_inches='tight', format='svg')

In [ ]:
I_factor=1000
a_dimer=0.22
a_monomer=0.3

monomer_fraction = 1 / (1 + np.exp(-np.arange(-50, 50)/8))
anisotropy = af.anisotropy_from_monomer(monomer_fraction, a_monomer, a_dimer, 1)
total_fluo = af.total_fluorescence_from_monomer(monomer_fraction, 1, I_factor)
I_parallel = af.intensity_parallel_from_anisotropy(anisotropy, total_fluo)
I_perpendicular = af.intensity_perpendicular_from_anisotropy(anisotropy, total_fluo)
anisotropy_from_int = af.anisotropy_from_intensity(I_parallel, 
                                                   I_perpendicular)

fig, axs = plt.subplots(3, 1, sharex=True, figsize=(5, 9))

axs[0].axhline(y=0, color='k', ls='--')
axs[0].axhline(y=1, color='k', ls='--')
axs[0].plot(monomer_fraction, c='b')
axs[0].set_ylabel('Fracción de Monómero')

axs[1].plot(I_parallel, c='g', label='Paralelo')
axs[1].plot(I_perpendicular, c='b', label='Perpendicular')
axs[1].legend()
axs[1].set_ylabel('Intensidad (u.a.)')

axs[2].axhline(y=a_dimer, color='k', ls='--')
axs[2].axhline(y=a_monomer, color='k', ls='--')
axs[2].plot(anisotropy, color='r')
axs[2].set_ylabel('Anisotropía')
axs[2].set_xlabel('Tiempo (min.)')

plt.subplots_adjust(hspace=0)
plt.savefig('mono_to_ani.svg', format='svg')

In [ ]:
from scipy.signal import savgol_filter
from scipy.interpolate import splrep, splev


def calculate_activity(ani, window_length=5, delta=5, delta_b=0):
    der = savgol_filter(ani, window_length=window_length,
                        polyorder=2,
                        deriv=1, delta=delta, mode='nearest')

    anisotropy_normalized = ani - np.min(ani)
    anisotropy_normalized = anisotropy_normalized / np.max(anisotropy_normalized)
    activity = der / ((1 + delta_b * anisotropy_normalized) ** 2)
    return activity

def interpolate(new_time, time, curve):
    """Interpolate curve using new_time as xdata"""
    if not np.isfinite(time).all():
        return np.array([np.nan])

    f = splrep(time, curve, k=3)
    return splev(new_time, f, der=0)

In [ ]:
I_factor=1000
a_dimer=0.22
a_monomer=0.3
time = np.arange(0, len(monomer_fraction))

monomer_fraction = 1 / (1 + np.exp(-np.arange(-50, 50)/6))
anisotropy = af.anisotropy_from_monomer(monomer_fraction, a_monomer, a_dimer, 1)
anisotropy_spaced = anisotropy[::5]
time = np.arange(0, len(monomer_fraction))
time_spaced = time[::5]

fig, axs = plt.subplots(3, 1, sharex=True, figsize=(3, 6))
time_maxs = []
for i in range(1000):
    anisotropy_exp = anisotropy_spaced + np.random.normal(0, 0.003, len(anisotropy_spaced))
    activity_exp = calculate_activity(anisotropy_exp)
    activity_interp = interpolate(time, time_spaced, activity_exp)
    time_maxs.append(np.where(activity_interp[:-10] == np.max(activity_interp[:-10]))[0][0])
    
    axs[0].plot(time_spaced, anisotropy_exp, alpha=0.3)
    
    axs[1].plot(time[:-10], activity_interp[:-10], alpha=0.3)

axs[0].plot(time[:-5], anisotropy[:-5], color='k', linewidth=3)
axs[0].plot(time[:-5], anisotropy[:-5], color='r')

activity_real = calculate_activity(anisotropy, window_length=25, delta=1)
axs[1].plot(time[:-10], activity_real[:-10], color='k', linewidth=3)
axs[1].plot(time[:-10], activity_real[:-10], color='r')

axs[2].axvline(x=50, color='k', alpha=0.7, linestyle='--')
axs[2].hist(time_maxs, bins=time)

axs[0].set_ylabel('Anisotropía')
axs[1].set_ylabel('Actividad')
axs[2].set_ylabel('Cuentas')
axs[2].set_xlabel('Tiempo (min.)')

axs[0].set_yticks([0.2, 0.25, 0.3])
axs[1].set_yticks([])

plt.subplots_adjust(hspace=0)
plt.savefig('savgol_test.svg', format='svg')
plt.show()

In [ ]:
np.std(time_maxs)

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True, figsize=(5, 8))

axs[0].plot(time, monomer_fraction)

axs[1].axvline(x=49.5, color='k', alpha=0.7, linestyle='--')
axs[1].plot(time[:-1], np.diff(monomer_fraction))

axs[2].axvline(x=49.5, color='k', alpha=0.7, linestyle='--')
axs[2].plot(time[:-1], np.diff(monomer_fraction))

axs[0].set_ylabel('Anisotropía')
axs[1].set_ylabel('Actividad')
axs[2].set_ylabel('Derivada de\nFracción Monómero')
axs[2].set_xlabel('Tiempo (min.)')

axs[1].set_yticks([])
axs[2].set_yticks([])

plt.subplots_adjust(hspace=0)
plt.savefig('monomer_ders.svg', format='svg')
plt.show()